# 자계추 hw2

20249433 최재필

In [1]:
import pandas as pd
import numpy as np

from scipy.stats.mstats import winsorize

## Load Data

### hw1 data

firm-level ME, BM, adjret

In [2]:
hw1_df = pd.read_csv('assignment1_data.csv')
hw1_df.head()

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM
0,10001,19870731,1987,3,4920.0,0.021277,0.016677,5.875,5.822125,5.822125,1.014415
1,10001,19870831,1987,3,4920.0,0.083333,0.078633,6.000,5.946000,5.822125,1.014415
2,10001,19870930,1987,3,4920.0,-0.022308,-0.026808,6.500,6.441500,5.822125,1.014415
3,10001,19871030,1987,3,4920.0,0.020000,0.014000,6.250,6.200000,5.822125,1.014415
4,10001,19871130,1987,3,4920.0,-0.029412,-0.032912,6.375,6.324000,5.822125,1.014415


In [3]:
hw1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1983365 entries, 0 to 1983364
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   permno       int64  
 1   date         int64  
 2   year         int64  
 3   exchcd       int64  
 4   siccd        float64
 5   retadj       float64
 6   eretadj      float64
 7   altprc_lag1  float64
 8   ME_lag1      float64
 9   ME_Jun       float64
 10  BM           float64
dtypes: float64(7), int64(4)
memory usage: 166.5 MB


### daily beta

In [4]:
beta_df = pd.read_csv('monthly_beta_data.csv')
beta_df.head()

,PERMNO,B_MKT,t
0,10000,0.7071,19861130
1,10000,0.6398,19861231
2,10000,0.6377,19870131
3,10000,0.3729,19870228
4,10000,0.3913,19870331


In [5]:
beta_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3225294 entries, 0 to 3225293
Data columns (total 3 columns):
 #   Column  Dtype  
---  ------  -----  
 0   PERMNO  int64  
 1   B_MKT   float64
 2   t       int64  
dtypes: float64(1), int64(2)
memory usage: 73.8 MB


### CPI

In [6]:
cpi_df = pd.read_excel('CPIAUCSL.xls', skiprows=10)
cpi_df.head()

,observation_date,CPIAUCSL
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95


In [7]:
cpi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 873 entries, 0 to 872
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   observation_date  873 non-null    datetime64[ns]
 1   CPIAUCSL          873 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 13.8 KB


## SAS 2

In [8]:
# * SAS 2: Add the beta measure to the monthly stock data set *******************************;
# * Keep only the last observation of each month to get monthly betas;

In [9]:
hw1_df

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM
0,10001,19870731,1987,3,4920.0,0.021277,0.016677,5.875000,5.822125,5.822125,1.014415
1,10001,19870831,1987,3,4920.0,0.083333,0.078633,6.000000,5.946000,5.822125,1.014415
2,10001,19870930,1987,3,4920.0,-0.022308,-0.026808,6.500000,6.441500,5.822125,1.014415
3,10001,19871030,1987,3,4920.0,0.020000,0.014000,6.250000,6.200000,5.822125,1.014415
4,10001,19871130,1987,3,4920.0,-0.029412,-0.032912,6.375000,6.324000,5.822125,1.014415
...,...,...,...,...,...,...,...,...,...,...,...
1983360,93436,20120831,2012,3,9999.0,0.040117,0.040017,27.420000,2890.945448,3295.556766,0.074994
1983361,93436,20120928,2012,3,9999.0,0.026648,0.026548,28.520000,3006.920688,3295.556766,0.074994
1983362,93436,20121031,2012,3,9999.0,-0.039228,-0.039328,29.280001,3097.004233,3295.556766,0.074994
1983363,93436,20121130,2012,3,9999.0,0.202215,0.202115,28.131399,3200.762464,3295.556766,0.074994


In [10]:
# * Generate the month-end date of each monthly stock observation;
# ** last trading date이지, 무조건 월말이 아니다. 예를들어 그 달은 내내 거래 안해서 5일이 마지막 거래일이다? 그럼 5일의 beta로 ;
# data monthly_stock_data1; 
#     set my_lib.assignment1_data; 
#     t = intnx('month', date, 0, 'end'); ** 날짜변수를 숫자변수인 t로 바꿔주는 과정 ;
#     format t yymmddn8.; 
# run; 

hw1_df['pddate'] = pd.to_datetime(hw1_df['date'].astype(str), format='%Y%m%d')
hw1_df['t'] = (hw1_df['pddate'] + pd.offsets.MonthEnd(0)).dt.strftime('%Y%m%d').astype(int)

In [11]:
# * Add beta information to the monthly stock data set;
# data monthly_stock_data2; 
#     merge monthly_stock_data1 (in = a) monthly_beta_data (in = b); 
#     by permno t; 
#     if a and b and missing(b_mkt) = 0; 
# run; 

monthly_stock_data2 = pd.merge(
    left=hw1_df, 
    right=beta_df, 
    left_on=['permno', 't'],
    right_on=['PERMNO', 't'],
    how='inner'
    )

monthly_stock_data2.dropna(subset=['B_MKT'], inplace=True)
monthly_stock_data2.drop(columns=['PERMNO'], inplace=True)

In [12]:
# NOTE: The data set WORK.MONTHLY_STOCK_DATA2 has 1972533 observations and 13 variables.
monthly_stock_data2.shape 

(1972533, 14)

## SAS 3

In [13]:
# * SAS 3: Generate Mktcap_CPI and Size variables ******************************************;

In [14]:
# * CPI in June of each year;
# data CPI_Jun; 
#     set CPI (where = (month(cpi_date) = 6)); 
#     t = year(cpi_date); 
#     keep t cpi; 
# run; 

CPI_Jun = cpi_df.loc[cpi_df['observation_date'].dt.month == 6, :].copy()
CPI_Jun['t'] = CPI_Jun['observation_date'].dt.year

In [15]:
# NOTE: There were 73 observations read from the data set WORK.CPI.
#       WHERE MONTH(cpi_date)=6;

CPI_Jun.shape

(73, 3)

In [16]:
# * CPI in Dec, 2012;
# %let CPI_2012 = 231.221; 

CPI_2012 = cpi_df[ cpi_df['observation_date'] == '2012-12-01']['CPIAUCSL'].values[0]
CPI_2012

231.221

In [17]:
# * Calculate Mktcap_CPI, Size, and log_BM variables;
# data monthly_stock_data2; 
#     set monthly_stock_data2; ** beta까지 넣어둔 assignment 1 data를 가져온다. ;
#     if month(date) > 6 then t = year(date); 
#     else t = year(date) - 1; 
# run; 

monthly_stock_data2['t'] = monthly_stock_data2['pddate'].apply(
    lambda x: x.year if x.month > 6 else x.year - 1
    )

In [18]:
# proc sort data = monthly_stock_data2; 
#     by t date permno; 
# run; 

monthly_stock_data2.sort_values(by=['t', 'pddate', 'permno'], inplace=True)

In [19]:
monthly_stock_data2.head()

,permno,date,year,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,ME_Jun,BM,pddate,t,B_MKT
10250,10137,19630228,1963,1,4911.0,-0.004866,-0.007166,51.375,479.73975,393.070500,0.443575,1963-02-28,1962,1.0588
59404,10823,19630228,1963,1,4925.0,-0.047771,-0.050071,39.250,199.74325,173.662125,0.347801,1963-02-28,1962,0.8783
96857,11404,19630228,1963,1,4932.0,-0.024602,-0.026902,86.375,1390.11925,1124.218875,0.630464,1963-02-28,1962,0.7534
105364,11543,19630228,1963,1,2011.0,-0.065789,-0.068089,9.500,14.63000,10.929750,1.362077,1963-02-28,1962,1.0872
114688,11674,19630228,1963,1,4931.0,-0.075540,-0.077840,34.750,998.22850,723.536125,0.531657,1963-02-28,1962,0.4470


In [20]:
monthly_stock_data2.shape

(1972533, 14)

In [21]:
# data monthly_stock_data3; 
#     merge monthly_stock_data2 (in = a) CPI_Jun (in = b); 
#     by t; 
#     if a; 

monthly_stock_data3 = pd.merge(
    left=monthly_stock_data2, 
    right=CPI_Jun, 
    on='t',
    how='left'
    )


In [22]:
#     ME_Jun_CPI = (ME_Jun / cpi) * &CPI_2012; ** cpi-adjusted ME ;
#     size = log(ME_Jun); ** size도 log 씌워줌 ;
#     size_CPI = log(ME_Jun_CPI); 
#     log_BM = log(BM); 
#     keep permno date year exchcd siccd retadj eretadj altprc_lag1 ME_lag1 
#     b_mkt size size_CPI BM log_BM; ** 이 5개가 중요한 데이터 ;
# run; 

monthly_stock_data3['ME_Jun_CPI'] = (monthly_stock_data3['ME_Jun'] / monthly_stock_data3['CPIAUCSL']) * CPI_2012
monthly_stock_data3['size'] = np.log(monthly_stock_data3['ME_Jun'])
monthly_stock_data3['size_CPI'] = np.log(monthly_stock_data3['ME_Jun_CPI'])
monthly_stock_data3['log_BM'] = np.log(monthly_stock_data3['BM'])

monthly_stock_data3 = monthly_stock_data3[['permno', 'date', 't', 'exchcd', 'siccd', 'retadj', 'eretadj', 'altprc_lag1', 'ME_lag1', 
                                           'B_MKT', 'size', 'size_CPI', 'BM', 'log_BM']]

In [23]:
monthly_stock_data3.head()

,permno,date,t,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,B_MKT,size,size_CPI,BM,log_BM
0,10137,19630228,1962,1,4911.0,-0.004866,-0.007166,51.375,479.73975,1.0588,5.973989,8.009190,0.443575,-0.812889
1,10823,19630228,1962,1,4925.0,-0.047771,-0.050071,39.250,199.74325,0.8783,5.157112,7.192313,0.347801,-1.056125
2,11404,19630228,1962,1,4932.0,-0.024602,-0.026902,86.375,1390.11925,0.7534,7.024844,9.060045,0.630464,-0.461299
3,11543,19630228,1962,1,2011.0,-0.065789,-0.068089,9.500,14.63000,1.0872,2.391488,4.426689,1.362077,0.309011
4,11674,19630228,1962,1,4931.0,-0.075540,-0.077840,34.750,998.22850,0.4470,6.584150,8.619351,0.531657,-0.631756


In [24]:
monthly_stock_data3.shape

(1972533, 14)

## SAS 4

In [25]:
# * SAS 4: Winsorize stock characteristic variables ****************************************;

In [26]:
# * Rename characteristic variables;
# data monthly_stock_data3; 
#     set monthly_stock_data3; 
#     ** _o : original data ;
#     rename b_mkt = b_mkt_o size = size_o size_CPI = size_CPI_o BM = BM_o log_BM = log_BM_o; 
# run; 




# 그냥 winsorize하면 되므로 SAS의 이런 과정 필요 없음. 

# monthly_stock_data3 = monthly_stock_data3.rename(
#     columns={
#         'B_MKT': 'b_mkt_o', 
#         'size': 'size_o', 
#         'size_CPI': 'size_CPI_o', 
#         'BM': 'BM_o', 
#         'log_BM': 'log_BM_o'
#         }
#     )

In [27]:
# * Calculate 0.5% and 99.5% level of each characteristic variable on a monthly basis;
# proc sort data = monthly_stock_data3; 
#     by date; 
# run; 

# proc univariate data = monthly_stock_data3 noprint; 
#     by date; 
#     var b_mkt_o size_o size_CPI_o BM_o log_BM_o; ** _o 붙은게 winsorize 된 것들 ;
#     output out = bounds pctlpts = 0.5 99.5 pctlpre = b_mkt_ size_ size_CPI_ BM_ log_BM_; 
# run; 

# * Merge the bounds with the monthly stock data and winsorize characteristic variables;
# data monthly_stock_data4; 
#     merge monthly_stock_data3 bounds; 
#     by date; 

#     array original(5) b_mkt_o size_o size_CPI_o BM_o log_BM_o; 
#     array winsorized(5) b_mkt size size_CPI BM log_BM; 
#     array l_bound(5) b_mkt_0_5 size_0_5 size_CPI_0_5 BM_0_5 log_BM_0_5; 
#     array u_bound(5) b_mkt_99_5 size_99_5 size_CPI_99_5 BM_99_5 log_BM_99_5; 

#     do ii = 1 to 5; 
#         if original(ii) < l_bound(ii) then winsorized(ii) = l_bound(ii); 
#         else if original(ii) > u_bound(ii) then winsorized(ii) = u_bound(ii); 
#         else winsorized(ii) = original(ii); 
#     end; 

#     drop b_mkt_0_5--log_BM_99_5 ii b_mkt_o size_o size_CPI_o BM_o log_BM_o; 
# run; 


WINSORIZE_LEVEL = 0.005

winsorize_cols = ['B_MKT', 'size', 'size_CPI', 'BM', 'log_BM']
monthly_stock_data4 = monthly_stock_data3.copy()
for col in winsorize_cols:
    monthly_stock_data4[f'{col}_o'] = monthly_stock_data4[col]

monthly_stock_data4[winsorize_cols] = monthly_stock_data4.groupby('date').transform(
    lambda x: winsorize(x, limits=(WINSORIZE_LEVEL, WINSORIZE_LEVEL))
    )[winsorize_cols]

In [28]:
monthly_stock_data4[monthly_stock_data4['BM'] != monthly_stock_data4['BM_o']].head()

,permno,date,t,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,B_MKT,size,size_CPI,BM,log_BM,B_MKT_o,size_o,size_CPI_o,BM_o,log_BM_o
2583,22138,19640731,1964,1,4011.0,-0.141935,-0.144935,38.750,49.05750,1.3222,3.892993,5.902057,4.004267,1.387360,1.3222,3.892993,5.902057,5.115054,1.632188
2730,26569,19640731,1964,1,4411.0,0.055901,0.052901,20.125,75.95175,-0.1315,4.330098,6.339163,4.004267,1.387360,-0.1315,4.330098,6.339163,7.530083,2.018906
2834,38914,19640731,1964,1,3573.0,-0.055118,-0.058118,95.250,440.05500,1.8883,6.086900,8.095964,0.108901,-2.217313,1.8883,6.086900,8.095964,0.071002,-2.645044
2843,39570,19640731,1964,1,2834.0,-0.122222,-0.125222,31.500,106.21800,1.1904,4.665494,6.674558,0.108901,-2.217313,1.1904,4.665494,6.674558,0.094043,-2.364002
3036,22138,19640831,1964,1,4011.0,0.045113,0.042313,33.250,42.09450,1.3108,3.892993,5.902057,4.004267,1.387360,1.3108,3.892993,5.902057,5.115054,1.632188


## SAS 5

In [ ]:
# * SAS 5: Calculate summary statistics;

In [38]:
# %let varlist = b_mkt size size_CPI BM log_BM; 

# ods exclude all; 

# proc sort data = monthly_stock_data4; 
#     by date permno; 
# run; 

# proc means data = monthly_stock_data4 mean std skew kurt min p5 p25 median p75 max n stackodsoutput nolabels; 
#     by date; 
#     var &varlist; 
#     ods output summary = stats_by_month; 
# run; 

# ods exclude none; 

def agg_pct(p):
    def percentiles(x):
        return np.percentile(x, p)

    percentiles.__name__ = f'p{p}'

    return percentiles

stats = [
    'mean',
    'std',
    'skew',
    # 'kurtosis', # Not a method of DataFrameGroupBy
    pd.Series.kurt,
    'count',

    'min',
    agg_pct(5),
    agg_pct(25),
    'median',
    agg_pct(75),
    'max',
]

summary_stats_df = monthly_stock_data4.groupby('date').agg(
    {'B_MKT': stats, 
     'size': stats, 
     'size_CPI': stats, 
     'BM': stats, 
     'log_BM': stats}
    )

summary_stats_df.head()

B_MKT                                                       \
              mean       std      skew      kurt count     min       p5   
date                                                                      
19630228  0.749081  0.314868  0.247166 -0.766816    73  0.2328  0.28536   
19630329  0.751323  0.313025  0.227727 -0.763642    73  0.2152  0.28174   
19630430  0.751832  0.311964  0.250390 -0.714297    73  0.2274  0.28682   
19630531  0.755737  0.315102  0.275094 -0.661578    73  0.2209  0.28182   
19630628  0.590955  0.234681  0.446181  0.059083    73  0.1511  0.25268   

                                  ...    log_BM                            \
             p25  median     p75  ...       std      skew      kurt count   
date                              ...                                       
19630228  0.4890  0.7534  1.0367  ...  0.305932  0.868193  3.108214    73   
19630329  0.4977  0.7647  1.0252  ...  0.305932  0.868193  3.108214    73   
19630430  0.4989  0.7652  1.0124  ...  0.305932  0.868193  3.108214    73   
19630531  0.5091  0.7646  1.0303  ...  0.305932  0.868193  3.108214    73   
19630628  0.4227  0.5749  0.7418  ...  0.305932  0.868193  3.108214    73   

                                                                     
               min       p5       p25    median       p75       max  
date                                                                 
19630228 -1.439968 -1.26034 -1.001794 -0.782376 -0.655325  0.309011  
19630329 -1.439968 -1.26034 -1.001794 -0.782376 -0.655325  0.309011  
19630430 -1.439968 -1.26034 -1.001794 -0.782376 -0.655325  0.309011  
19630531 -1.439968 -1.26034 -1.001794 -0.782376 -0.655325  0.309011  
19630628 -1.439968 -1.26034 -1.001794 -0.782376 -0.655325  0.309011  

[5 rows x 55 columns]

In [39]:
# * Calculate the time-series-means of the summary statistics for the variables in the "varlist";

winsorize_cols

['B_MKT', 'size', 'size_CPI', 'BM', 'log_BM']

In [54]:
# proc sort data = stats_by_month; ** cross sectional로 구해놓은 bm mean들을 한 번 더 time-series로 mean ;
#     by variable date; 
# run; 

# proc means data = stats_by_month mean nolabels noprint; 
#     by variable; 
#     var mean stddev skew kurt min p25 median p75 max n; 
#     output out = stats (drop = _TYPE_ _FREQ_) mean(mean stddev skew kurt min p25 median p75 max n) = mean stddev skew kurt min p25 median p75 max n; 
# run; 

ts_summary_stats_df = pd.DataFrame(summary_stats_df.mean(), columns=['ts_mean'])
ts_summary_stats_df.index.set_names(['var', 'stats'], inplace=True)

ts_summary_stats_df = ts_summary_stats_df.unstack(level='stats')
ts_summary_stats_df.columns = ts_summary_stats_df.columns.droplevel(0) # ts_mean이라는 level(0)을 제거하고 남은 columns들을 반환
ts_summary_stats_df

stats,mean,std,skew,kurt,count,min,p5,p25,median,p75,max
var,,,,,,,,,,,
B_MKT,0.809365,0.586924,0.489271,0.340399,3293.043406,-0.567353,-0.019651,0.392367,0.746069,1.162343,2.609258
size,4.593019,1.911168,0.300007,-0.298195,3293.043406,0.521517,1.680205,3.185155,4.455764,5.883860,9.943393
size_CPI,5.472245,1.911168,0.300007,-0.298195,3293.043406,1.400743,2.559431,4.064380,5.334989,6.763086,10.822619
BM,0.885181,0.724305,2.213206,8.062992,3293.043406,0.043320,0.154878,0.412682,0.714026,1.131844,4.960526
log_BM,-0.488620,0.827443,-0.569573,0.915820,3293.043406,-3.403004,-1.969685,-0.958599,-0.400904,0.061488,1.489646


## SAS 6

In [ ]:
# * SAS 6: Calculate correlations;

In [57]:
monthly_stock_data4

,permno,date,t,exchcd,siccd,retadj,eretadj,altprc_lag1,ME_lag1,B_MKT,size,size_CPI,BM,log_BM,B_MKT_o,size_o,size_CPI_o,BM_o,log_BM_o
0,10137,19630228,1962,1,4911.0,-0.004866,-0.007166,51.375000,479.739750,1.0588,5.973989,8.009190,0.443575,-0.812889,1.0588,5.973989,8.009190,0.443575,-0.812889
1,10823,19630228,1962,1,4925.0,-0.047771,-0.050071,39.250000,199.743250,0.8783,5.157112,7.192313,0.347801,-1.056125,0.8783,5.157112,7.192313,0.347801,-1.056125
2,11404,19630228,1962,1,4932.0,-0.024602,-0.026902,86.375000,1390.119250,0.7534,7.024844,9.060045,0.630464,-0.461299,0.7534,7.024844,9.060045,0.630464,-0.461299
3,11543,19630228,1962,1,2011.0,-0.065789,-0.068089,9.500000,14.630000,1.0872,2.391488,4.426689,1.362077,0.309011,1.0872,2.391488,4.426689,1.362077,0.309011
4,11674,19630228,1962,1,4931.0,-0.075540,-0.077840,34.750000,998.228500,0.4470,6.584150,8.619351,0.531657,-0.631756,0.4470,6.584150,8.619351,0.531657,-0.631756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1972528,93428,20121231,2012,3,9999.0,0.148593,0.148493,31.629999,880.421027,1.5204,6.682734,6.694466,0.163951,-1.808187,1.5204,6.682734,6.694466,0.163951,-1.808187
1972529,93429,20121231,2012,3,9999.0,0.007672,0.007572,29.980000,2616.414520,0.4912,7.789740,7.801473,0.112264,-2.186904,0.4912,7.789740,7.801473,0.112264,-2.186904
1972530,93433,20121231,2012,3,9999.0,-0.517704,-0.517804,0.850100,39.250816,1.8107,3.354164,3.365897,1.187065,0.171484,1.8107,3.354164,3.365897,1.187065,0.171484
1972531,93434,20121231,2012,3,9999.0,0.037634,0.037534,7.440000,58.575120,0.3931,3.593419,3.605152,0.569317,-0.563317,0.3931,3.593419,3.605152,0.569317,-0.563317


In [62]:
# proc corr data = monthly_stock_data4 outp = pcorr_by_month (where = (_TYPE_ = "CORR")) noprint; 
#     by date; 
#     var &varlist; 
# run; 

monthly_corr = monthly_stock_data4.groupby('date')[winsorize_cols].corr()


In [68]:
# * Calculate the time-series-means of the correlations for variables in the "varlist";
# proc sort data = pcorr_by_month; 
#     by _name_ date; 
# run; 

# proc means data = pcorr_by_month mean nolabels noprint; 
#     by _name_; 
#     var &varlist; 
#     output out = pcorr (keep = _NAME_ &varlist) mean(&varlist) = &varlist; 
# run; 

ts_monthly_corr = pd.DataFrame(monthly_corr.unstack().mean(), columns=['ts_mean'])
ts_monthly_corr.index.set_names(['var1', 'var2'], inplace=True)
ts_monthly_corr = ts_monthly_corr.unstack(level='var2')
ts_monthly_corr.columns = ts_monthly_corr.columns.droplevel(0)

ts_monthly_corr

var2,B_MKT,size,size_CPI,BM,log_BM
var1,,,,,
B_MKT,1.000000,0.297880,0.297880,-0.193636,-0.220787
size,0.297880,1.000000,1.000000,-0.290795,-0.251758
size_CPI,0.297880,1.000000,1.000000,-0.290795,-0.251758
BM,-0.193636,-0.290795,-0.290795,1.000000,0.852688
log_BM,-0.220787,-0.251758,-0.251758,0.852688,1.000000


## SAS 7

In [69]:
# * SAS 7: Dependent-sort stocks into 25 portfolios based on size and BM *******************************;

In [71]:
winsorize_cols

['B_MKT', 'size', 'size_CPI', 'BM', 'log_BM']

In [73]:
# * Calculate size breakpoints as 20th, 40th, 60th, and 80th size percentiles among NYSE stocks in each month;
# *** NYSE stock들만 가지고 breakpoint를 찾아라. ;
# proc univariate data = monthly_stock_data4 (where = (exchcd in (1, 31))) noprint; 
#     by date; 
#     var size; 
#     output out = size_breakpoints pctlpts = 20 40 60 80 pctlpre = size_; 
# run; 

# * Merge the size breakpoints with the monthly stock data and define size sorted portfolios;
# data monthly_stock_data5; 
#     merge monthly_stock_data4 size_breakpoints; 
#     by date; 

#     if size < size_20 then p1 = 1; 
#     else if size < size_40 then p1 = 2; 
#     else if size < size_60 then p1 = 3; 
#     else if size < size_80 then p1 = 4; 
#     else p1 = 5; 
# run; 

monthly_stock_data4 = monthly_stock_data4[ monthly_stock_data4['exchcd'].isin([1, 31]) ].copy()
monthly_stock_data4['p1'] = monthly_stock_data4.groupby('date')['size'].transform(
    lambda x: pd.qcut(x, q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=False, duplicates='drop') + 1
    )


In [75]:
# proc univariate data = monthly_stock_data5 noprint; 
#     by date p1; 
#     var BM; 
#     output out = BM_breakpoints pctlpts = 20 40 60 80 pctlpre = BM_; 
# run; 

# * Merge the BM breakpoints with the monthly stock data and define BM sorted portfolios in each size sorted portfolio;
# data monthly_stock_data6; 
#     merge monthly_stock_data5 BM_breakpoints; 
#     by date p1; 

#     if BM < BM_20 then p2 = 1; 
#     else if BM < BM_40 then p2 = 2; 
#     else if BM < BM_60 then p2 = 3; 
#     else if BM < BM_80 then p2 = 4; 
#     else p2 = 5; 
# run; 


## 중요한 차이: independent sort가 아니라 dependent sort임.

monthly_stock_data4['p2'] = monthly_stock_data4.groupby(['date', 'p1'])['BM'].transform(
    lambda x: pd.qcut(x, q=[0, 0.2, 0.4, 0.6, 0.8, 1], labels=False, duplicates='drop') + 1
    )

In [89]:
# * Calculate the time-series average number of stocks in each portfolio;
# proc sort data = monthly_stock_data6; 
#     by date p1 p2; 
# run; 

# proc means data = monthly_stock_data6 n nolabels noprint; 
#     by date p1 p2; ** 서로 다른 permno들이 각 p1, p2에 몇개씩 있는지 count ;
#     var permno; 
#     output out = nstocks_per_p n = nstocks; 
# run; 

# proc sort data = nstocks_per_p; 
#     by p1 p2; 
# run; 

nstocks_per_p = pd.DataFrame(monthly_stock_data4.groupby(['date', 'p1', 'p2']).size(), columns=['nstocks'])
nstocks_per_p.index.set_names(['date', 'p1', 'p2'], inplace=True)
nstocks_per_p = nstocks_per_p.unstack(level=['p1', 'p2'])
nstocks_per_p.columns = nstocks_per_p.columns.droplevel(0)

ts_nstocks_per_p = pd.DataFrame(nstocks_per_p.mean(), columns=['ts_mean'])
ts_nstocks_per_p = ts_nstocks_per_p.unstack(level='p2')
ts_nstocks_per_p.columns = ts_nstocks_per_p.columns.droplevel(0)
ts_nstocks_per_p

p2,1,2,3,4,5
p1,,,,,
1,45.699499,45.128548,45.090150,45.128548,45.507513
2,45.579299,44.993322,45.000000,44.993322,45.392321
3,45.589316,45.006678,44.961603,45.006678,45.393990
4,45.580968,44.994992,45.000000,44.994992,45.393990
5,45.654424,45.088481,45.040067,45.088481,45.474124
